# Get PIREPS data

## Data Dictionary

### Intensities 
LGT - Light  
MOD - Moderate  
SEV - Severe  
EXT - Extreme

### Frequencies
INTMT - Intermittent 
OCL - Occasional  
CONS - Constant  

### Turbulence types
CAT - Clear Air Turbulence  
CHOP - 

In [ ]:
import requests
import os
import numpy as np
import pandas as pd

## Download PIREPs from January 2025

In [ ]:

BASE_URL = "https://mesonet.agron.iastate.edu/cgi-bin/request/gis/pireps.py"

year = 2025
month = 1
query = {"year1": year, 
         "month1": month, 
         "year2": year, 
         "month2": month + 1,
         "artcc": "_ALL", 
         "fmt": "csv"}
print(f"Performing GET request, this may take a moment...")
r = requests.get(f"{BASE_URL}", params=query, stream=True)
dirname = os.path.join("raw_pirep_data", str(year))
os.makedirs(dirname, exist_ok=True)
filename = os.path.join(dirname, f"{month:02}_raw_pireps.csv")
with open(filename, 'w') as file:
    file.write(r.text)
print(f"Finished writing file to {filename}")

In [ ]:
# This chunk converts our pireps csv to a pandas DF.
import pandas as pd

pireps = pd.read_csv(filename, on_bad_lines="skip")
print(pireps.columns)


### Building new columns
This section will add to our existing PIREPS DF. First, we need to build a time column in a datetime format, so that it's compatible with our NEXRAD DataFrame.

In [ ]:
pireps['datetime'] = pd.to_datetime(pireps['VALID'], format='%Y%m%d%H%M')

# Display the first few rows to confirm the conversion
print(pireps[['VALID', 'datetime']].head(5))

Next, we want to have a turbulence intensity category. We can do this by extracting key information from our TURB column.

In [ ]:
def get_turb_intensity(row):

    turb_list = str(row['TURBULENCE']).replace('-', ' ').split()

     # Handle cases where PIREP contains two levels of turbulence without a dash
    if 'LGT' in turb_list and 'MOD' in turb_list:
         return 2
    
    if 'MOD' in turb_list and 'SEV' in turb_list:
         return 4
    
    if 'SEV' in turb_list and 'EXTRM' in turb_list:
         return 6

    # Define the turbulence intensity map
    turbulence_map = {
        'NONE': 0,
        'NEG': 0,
        'LGT': 1,
        'MOD': 3,
        'SEV': 5,
        'EXTRM': 7
    }

    # Check for NaN
    if 'nan' in turb_list:
            return np.nan
        
    # Look for turbulence intensity in the map
    for key in turbulence_map:
        if key in turb_list:
            return turbulence_map[key]
        
   
        
     # Return NaN if no known turbulence level is found
    return np.nan
pireps['turbulence_intensity'] = pireps.apply(get_turb_intensity, axis = 1)


In [ ]:
pireps['turbulence_intensity'] = pireps.apply(get_turb_intensity, axis = 1)
print(pireps['turbulence_intensity'].unique())

In [ ]:
subset = pireps[pireps['turbulence_intensity'] == 0]
print(subset.URGENT)

In [ ]:
pireps.head()

In [ ]:
only_turb_pireps = pireps.dropna(subset=['turbulence_intensity'])

In [ ]:
# Sort the pireps into 2 sets - 1 with pireps for turbulence and 1 without
turb_pireps = pireps[~np.isnan(pireps['turbulence_intensity'])]
non_turb_pireps = pireps[np.isnan(pireps['turbulence_intensity'])]
len(non_turb_pireps.index)
turb_pireps

In [ ]:
clean_turb_pireps = turb_pireps.drop(["ICING", "ATRCC", "PRODUCT_ID", "VALID"], axis=1)

clean_turb_pireps